In [1]:
import json
import requests
from lxml import etree
import requests
import time
import os

In [2]:
s = requests.Session()
s.headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

In [3]:
with open('fund.ini','r') as f:
    list=[]
    for line in f.readlines():
        kv = line.strip().split()
        list.append(kv)

In [4]:
def str_rate(new,old):
    s = ''
    if new > old:
        s = '+'
    return s + format((new-old)/old*100,'.2f')+'%'

In [5]:
code = '502023'
code = str(code)
url = 'https://fund.10jqka.com.cn/data/client/myfund/'+code
res = s.get(url).json()['data'][0]
hqcode = res['hqcode']
url = 'https://gz-fund.10jqka.com.cn/?module=api&controller=index&action=chart&info=vm_fd_'+hqcode
#print(url)
gz_all_raw = s.get(url).content.decode().split(';')[1:]
date_gz = gz_all_raw[0].split('~')[0].split('|')[1]
gz_all_raw[0] = gz_all_raw[0].split('~')[-1]
#gz_all_raw

In [6]:
def get(code):
    code = str(code)
    url = 'https://fund.10jqka.com.cn/data/client/myfund/'+code
    res = s.get(url).json()['data'][0]
    
    name = res['name']
    net = float(res['net'])
    enddate = res['enddate']
    net1 = float(res['net1'])
    enddate1 = res['enddate1']
    hqcode = res['hqcode']
    
    url = 'https://gz-fund.10jqka.com.cn/?module=api&controller=index&action=chart&info=vm_fd_'+hqcode
    #print(url)
    gz_all_raw = s.get(url).content.decode().split(';')[1:]
    gzdate = gz_all_raw[0].split('~')[0].split('|')[1][5:]
    gz_all_raw[0] = gz_all_raw[0].split('~')[-1]

    gz_all = []
    for i in gz_all_raw:
        gz_all.append(float(i.split(',')[1]))
            
    return name+' '+code,gz_all,net,enddate,net1,gzdate


In [7]:
STATE = 0
# 0 还没开盘
# 1 交易中
# 2 收盘了
# 3 中午休市
# 4 净值更新了
# 5 今天可能休市吧

def state(gzdate,enddate,gz_all):
    if gzdate == time.strftime('%m-%d'):
        if gzdate == enddate:
            s = '净值更新了!'
            STATE = 4
        elif len(gz_all) == 121:
            s = '中午休市!'
            STATE = 3
        elif len(gz_all) == 242:
            s = '收盘了!'
            STATE = 2
        elif len(gz_all) > 0 :
            s = '交易中!'
            STATE = 1
        else:
            s = '还没开盘!'
            STATE = 0
    else:
        s = '今天可能休市吧!'
        STATE = 5
            
    return STATE,s

In [8]:
def cal(gz_all,net,net1,share,STATE):
    if STATE in [0, 4, 5]:
        gz_now,gz_max,gz_min = (0,0,0)
        amount = net * share
        delta = (net - net1)*share
        
    elif STATE in [1, 2, 3]:
        gz_now = gz_all[-1]
        gz_max = max(gz_all)
        gz_min = min(gz_all)
        amount = gz_now * share
        delta = (gz_now - net)*share
    
    return amount, delta, gz_now, gz_max, gz_min

In [9]:
def str_final(name,gz_all,net,enddate,net1,share,STATE):
    
    amount, delta, gz_now, gz_max, gz_min = cal(gz_all,net,net1,share,STATE)
    
    if STATE in [1, 2, 3]:
        gz_now = gz_all[-1]
        gz_max = max(gz_all)
        gz_min = min(gz_all)
        amount = gz_now * share
        delta = (gz_now - net)*share
        gz_str = '盘中估值 '+str(gz_now)+' '+str_rate(gz_now,net)+'\n'
        jz_str = '单位净值('+enddate[5:]+') '+str(net)+' '+str_rate(net,net1)+'\n'
        gz_max_min_str = '目前最高/低估值 '+str(gz_max)+'/'+str(gz_min)+' '+str_rate(gz_max,net)+'/'+str_rate(gz_min,net)+'\n'
        zjz_str = '总估值 '+format(gz_now*share,'.4f')+'元 '+ str_rate(gz_now,net)+'\n'
    else:
        amount = net * share
        delta = (net - net1)*share
        gz_str = ''
        gz_max_min_str = ''
        jz_str = '单位净值('+enddate[5:]+') '+str(net)+' '+str_rate(net,net1)+'\n'
        zjz_str = '总净值 '+format(net*share,'.4f')+'元 ('+enddate+') '+ str_rate(gz_now,net)+'\n'
    
    if delta < 0:
        s = 'T_T\n'
    else:
        s = '^_^\n'
    return s + name+'\n' + gz_max_min_str + gz_str + jz_str + zjz_str + '\n'

def wirte_front(fname,str):
    f = open(fname,'r')
    with open('tmp','w') as t:
        t.write(str)
        t.write(f.read())
    f.close()    
    t.close()
    os.remove(fname)
    os.rename('tmp',fname)

In [19]:
def td(new,ori,date=None):
    if new > ori:
        c = 'red'
    elif new < ori:
        c = 'green'
    else:
        c = 'gray'
    if date is not None:
        return '<td style:"text-align: center;"><span style="color:%s;font-size:20;font-weight:bold;">%s</span><br><span style="color:%s;font-size:15;font-weight:bold;">%s</span><br><span style="font-size:10;color=gray;">(%s)</span></td>'%(c,format(new,'.2f'),c,str_rate(new,ori),date[-5:])
    return '<td style:"text-align: center;"><span style="color:%s;font-size:20;font-weight:bold;">%s</span><br><span style="color:%s;font-size:15;font-weight:bold;">%s</span></td>'%(c,format(new,'.2f'),c,str_rate(new,ori))

In [11]:
def html_str(name,gz_all,net,enddate,net1,share,STATE):
    
    amount, delta, gz_now, gz_max, gz_min = cal(gz_all,net,net1,share,STATE)
    s='<tr>'
    s+='<td>'+name.split()[0]+'</td>'
    s+='<td>'+name.split()[1]+'</td>'
    
    if STATE in [1, 2, 3]:
        gz_now = gz_all[-1]
        gz_max = max(gz_all)
        gz_min = min(gz_all)
        amount = gz_now * share
        delta = (gz_now - net)*share
        gz_str = td(gz_now,net)
        jz_str = td(net,net1,enddate)
        gz_max_str = td(gz_max,net)
        gz_min_str = td(gz_min,net)
        zjz_str = td(gz_now*share,net*share,'---')
        #gz_str = '<td>'+str(gz_now)+' '+str_rate(gz_now,net)+'</td>'
        #jz_str = '<td>'+str(net)+' '+str_rate(net,net1)+'</td>'
        #gz_max_str = '<td>'+str(gz_max)+' '+str_rate(gz_max,net)+'</td>'
        #gz_min_str = '<td>'+str(gz_min)+' '+str_rate(gz_min,net)+'</td>'
        #zjz_str = '<td><span class="%s val">%s</span><br><span class="%s rate">%s</span><span>%s</span></td>'%(c1,format(gz_now*share,'.4f')+'元',c2,str_rate(gz_now,net),'(估)')
    else:
        amount = net * share
        delta = (net - net1)*share
        gz_str = '<td>---</td>'
        gz_max_str = '<td>---</td>'
        gz_min_str = '<td>---</td>'
        jz_str = td(net,net1,enddate)
        zjz_str = td(net*share,net1*share)
        #zjz_str = '<td><span style="color:$color$;">%s</span><br><span>%s</span><span>%s</span></td>'%(format(net*share,'.4f')+'元',str_rate(gz_now,net),'('+enddate+')')
    
    if delta < 0:
        delta = -1
    return s + zjz_str + gz_max_str + gz_min_str + gz_str + jz_str + '</tr>'

def wirte_front(fname,str):
    f = open(fname,'r')
    with open('tmp','w') as t:
        t.write(str)
        t.write(f.read())
    f.close()    
    t.close()
    os.remove(fname)
    os.rename('tmp',fname)

In [21]:
def main_html():
    amount_ori = 0
    amount_now = 0
    jz = True
    s=''
    with open('fund.html','w') as  f:        
        fstr='<table border="1" style="text-align: center;width:900;"><tr><th>基金名</th><th>代码</th><th>最新值</th><th>最高估值</th><th>最低估值</th><th>盘中估值</th><th>单位净值</th></tr>'
        f.write(fstr)
        print(fstr)
        for j in list:
            share = float(j[1])
            name,gz_all,net,enddate,net1,gzdate = get(j[0])
            STATE,strState = state(gzdate,enddate,gz_all)
            amount, delta, gz_now, gz_max, gz_min = cal(gz_all,net,net1,share,STATE)
            if not STATE == 4:
                jz = False
            if STATE in [0, 5, 4]:
                amount_now += net * share
                amount_ori += net1 * share
            elif STATE in [1, 2, 3]:
                amount_now += gz_now * share
                amount_ori += net * share
            fstr = html_str(name,gz_all,net,enddate,net1,share,STATE)
            print(fstr)
            f.write(fstr)
        fstr = '</table>'
        f.write(fstr)
        print(fstr)
        f.write('</html>')    
    if amount_now > amount_ori:
        color = 'red'
    elif amount_now < amount_ori:
        color = 'green'
    else:
        color = 'gray'
    s1 = format(amount_now-amount_ori,'.2f')+' 元 '
    s2 = str_rate(amount_now,amount_ori)
    s3 = ' ('+gzdate+')'
    if not jz:
        s3 += '*'
    s += '<html><head><meta charset="utf-8"><style>td{text-align: center;}table{table-layout: fixed;width: max-content;}</style></head>'
    s += '<div><p><span style="color:$color$;font-size:40;font-weight:900;">%s </span> <span style="color:$color$;font-size:30;font-weight:400;">%s</span> <span style="color:black;font-size:25;">%s</span></p></div>'%(s1,s2,s3)
    s = s.replace('$color$',color)
    
    
    s += '<h2>'
    if jz:
        s += '净值更新了！'
    elif STATE in [0, 1, 2, 3, 5]:
        s += strState
    else:
        s += '收盘了！等待更新净值！'
    s += '  ' + time.strftime('%m-%d')
    s += '</h2>'
    s += '<br>'
    print(s)
    wirte_front('fund.html',s)

In [22]:
main_html()

<table border="1" style="text-align: center;width:900;"><tr><th>基金名</th><th>代码</th><th>最新值</th><th>最高估值</th><th>最低估值</th><th>盘中估值</th><th>单位净值</th></tr>
<tr><td>泰康中证港股通大消费A</td><td>006786</td><td style:"text-align: center;"><span style="color:red;font-size:20;font-weight:bold;">86.06</span><br><span style="color:red;font-size:15;font-weight:bold;">+0.55%</span><br><span style="font-size:10;color=gray;">(---)</span></td><td style:"text-align: center;"><span style="color:red;font-size:20;font-weight:bold;">1.61</span><br><span style="color:red;font-size:15;font-weight:bold;">+1.00%</span></td><td style:"text-align: center;"><span style="color:green;font-size:20;font-weight:bold;">1.58</span><br><span style="color:green;font-size:15;font-weight:bold;">-1.07%</span></td><td style:"text-align: center;"><span style="color:red;font-size:20;font-weight:bold;">1.60</span><br><span style="color:red;font-size:15;font-weight:bold;">+0.55%</span></td><td style:"text-align: center;"><span style="col

In [14]:
def main():
    amount_ori = 0
    amount_now = 0
    s = ''
    jz = True
    with open('fund.txt','w') as  f:
        for j in list:
            share = float(j[1])
            name,gz_all,net,enddate,net1,gzdate = get(j[0])
            STATE,strState = state(gzdate,enddate,gz_all)
            amount, delta, gz_now, gz_max, gz_min = cal(gz_all,net,net1,share,STATE)
            if not STATE == 4:
                jz = False
            if STATE in [0, 5, 4]:
                amount_now += net * share
                amount_ori += net1 * share
            elif STATE in [1, 2, 3]:
                amount_now += gz_now * share
                amount_ori += net * share
            fstr = str_final(name,gz_all,net,enddate,net1,share,STATE)
            print(fstr)
            f.write(fstr)
    
    if amount_now > amount_ori:
        s += '+'
    s += format(amount_now-amount_ori,'.2f')+' 元 '
    s += str_rate(amount_now,amount_ori)
    s += ' ('+gzdate+')'
    if not jz:
        s += '*'
        
    s += '\n'
    if jz:
        s += '净值更新了！'
    elif STATE in [0, 1, 2, 3, 5]:
        s += strState
    else:
        s += '收盘了！等待更新净值！'
    s += '  ' + time.strftime('%m-%d')
    s += '\n----------------------------------------\n'
    print(s)
    wirte_front('fund.txt',s)

In [15]:
main()

^_^
泰康中证港股通大消费A 006786
目前最高/低估值 1.61135/1.5783 +1.00%/-1.07%
盘中估值 1.60414 +0.55%
单位净值(07-06) 1.5954 -1.80%
总估值 86.0621元 +0.55%


^_^
鹏华钢铁 502023
目前最高/低估值 1.7162/1.66073 +3.26%/-0.08%
盘中估值 1.71454 +3.16%
单位净值(07-06) 1.662 -0.66%
总估值 51.4533元 +3.16%


^_^
广发中证全指家用电器指数A 005063
目前最高/低估值 1.48452/1.45184 +1.92%/-0.32%
盘中估值 1.47038 +0.95%
单位净值(07-06) 1.4565 +0.31%
总估值 39.8767元 +0.95%


^_^
创金合信科技成长股票A 005495
目前最高/低估值 2.1398/2.07873 +1.41%/-1.49%
盘中估值 2.1347 +1.17%
单位净值(07-06) 2.1101 +0.22%
总估值 30.8464元 +1.17%


^_^
信诚新兴产业混合 000209
目前最高/低估值 4.35696/4.05069 +5.80%/-1.63%
盘中估值 4.35175 +5.68%
单位净值(07-06) 4.118 +0.61%
总估值 4.3518元 +5.68%


+3.01 元 +1.44% (07-07)*
收盘了!  07-07
----------------------------------------

